### 'reanalysis-era5-single-levels-preliminary-back-extension'

In [3]:
from queue import Queue
from threading import Thread
import cdsapi
from time import time
import datetime
import os


def create_download_links(first_year,
                          last_year,
                          save_folder,
                          type_of_ERA,
                          variables,
                          location,
                          N,
                          W,
                          S,
                          E):

    download_links = [[[year] for year in range(
        first_year, last_year+1, 1)] for _ in range(len(variables))]

    for i, var in enumerate(download_links):
        for j, year in enumerate(var):
            year.append(save_folder)
            year.append(type_of_ERA)
            year.append(variables[i])
            year.append(location)
            year.append(N)
            year.append(W)
            year.append(S)
            year.append(E)

    return download_links


# variables = ['2m_temperature', 'evaporation', 'potential_evaporation',
#              'runoff', 'total_precipitation']

# type_of_ERA
# 'reanalysis-era5-single-levels'
# 'reanalysis-era5-single-levels-preliminary-back-extension'

# location
# location='CONUS',
# N=51,
# W=-125,
# S=25,
# E=-51)

# location='Russia',
# N=82,
# W=19,
# S=41,
# E=-169)


download_links = create_download_links(first_year=1979,
                                       last_year=2020,
                                       save_folder=r'E:/ERA5_data/',
                                       type_of_ERA='reanalysis-era5-single-levels',
                                       variables=[
                                           'mean_surface_direct_short_wave_radiation_flux'],
                                        location='CONUS',
                                        N=51,
                                        W=-125,
                                        S=25,
                                        E=-51)


In [ ]:
def download_request(link):
    filename = link[1] + '{}/{}/{}/{}_{}.nc'.format(
        link[4], link[2], link[3], link[3], link[0])

    try:
        os.makedirs(link[1] + '{}/{}/{}/'.format(link[4], link[2], link[3]))
    except FileExistsError:
        # directory already exists
        pass
        
    if (os.path.isfile(filename)):
        print('existing', filename)  # return existed file
    else:
        print(filename)
        c = cdsapi.Client()
        c.retrieve(link[2],
                   {
            'product_type': 'reanalysis',
            'variable': link[3],
            'year': link[0],
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '01:00', '02:00',
                '03:00', '04:00', '05:00',
                '06:00', '07:00', '08:00',
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ],
            'area': [link[5],
                     link[6],
                     link[7],
                     link[8]],
            'format': 'netcdf',
        },
            filename)

# download processing


class MultiDownload(Thread):

    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue

    def run(self):
        while True:
            # recieve links from queue
            link = self.queue.get()
            download_request(link)
            self.queue.task_done()


queue = Queue()

for request in range(4):
    worker = MultiDownload(queue)
    worker.daemon = True
    worker.start()

for var_link in download_links:
    for link in var_link:
        queue.put((link))

queue.join()
